In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer
from transformers import default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 10.3 MB/s 
     |████████████████████████████████| 298 kB 8.2 MB/s 
     |████████████████████████████████| 243 kB 62.2 MB/s 
     |████████████████████████████████| 61 kB 684 kB/s 
     |████████████████████████████████| 132 kB 67.6 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 160 kB 64.5 MB/s 
     |████████████████████████████████| 192 kB 76.3 MB/s 
     |████████████████████████████████| 271 kB 69.6 MB/s 
     |████████████████████████████████| 3.3 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 72.8 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 
     |████████████████████████████████| 3.3 MB 28.8 MB/s 
     |████████████████████████████████| 1.2 MB 8.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = '/content/drive/MyDrive/EMNLP_folder_4/headline_model/checkpoint-4518'
tokenizer = PegasusTokenizer.from_pretrained(model_path)
model = PegasusForConditionalGeneration.from_pretrained(model_path).to(device)

In [ ]:
src_text = ["""Boo. :(This is the best place at PDX to get breakfast."""]
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

['It is preferable to spend the money elsewhere']


In [ ]:
# reviews_path = '/content/drive/MyDrive/gen_datasets/res_reviews_final.json'
reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
data = json.load(f)
data = json.loads(data)

In [ ]:
x = {}
for key_ in data.keys():
  for a in data[key_]['review_list']:
    if(a['rating']!=None):
      x[a['text']] = a['rating']

In [ ]:
import collections
od = collections.OrderedDict(sorted(x.items()))

In [ ]:
reviews = []
for j in sorted.items():
  reviews.append(j[0])

In [ ]:
num_kp = int(len(reviews)/50)
input = []
for i in range(len(reviews)):
  if(i%num_kp==0):
    input.append(reviews[i])
    src_text = ''.join(input)
    batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(i,tgt_text)
    input = []

0 ['The food is not good enough']
2 ['The food is good/healthy']
4 ['Coffee is expensive']
6 ['The food is unhealthy/unhealthy']
8 ['There are issues more important to fund than food']
10 ['There are issues more important to fund than food']
12 ['Parents should be permitted to choose the education of their children']
14 ['A pre-made sandwich is unhealthy/unnatural']
16 ['People should choose for themselves whether or not to eat meat']
18 ['The Celiac Disease is a big concern for those who are affected']
20 ['Alcohol and tobacco are more harmful drugs yet remain legal']
22 ['It is preferable to spend the money elsewhere']
24 ['Alcohol and tobacco are more harmful drugs yet remain legal']
26 ['The food is good']
28 ['There are issues more important to fund than food/staff shortages.']
30 ['The airport is a good place to eat/drink/shop']
32 ['The food is good']
34 ['The bakery is a good choice for a quick snack/snack']
36 ['The airport is a good business location']
38 ['The food is good/h